### Example: Visualizing Seattle Bicycle Counts

• As a more involved example of working with some time series data, let’s take a look at bicycle counts on Seattle’s Fremont Bridge.

• This data comes from an automated bicycle counter, installed in late 2012, which has inductive sensors on the east and west sidewalks of the bridge.

• Once this dataset is downloaded, we can use Pandas to read the CSV output into a DataFrame.

• We will specify that we want the Date as an index, and we want these dates to be automatically parsed: